In [577]:
import time

import os, sys
import pandas as pd, numpy as np

In [7]:
__scrobbles__ = 'downloads/gps56-2.csv'

In [8]:
pd.options.mode.chained_assignment = None

In [9]:
weekday_map = {0: 'monday',
              1: 'tuesday', 
              2: 'wednesday',
              3: 'thursday', 
              4: 'friday', 
              5: 'saturday', 
              6: 'sunday'}

In [10]:
df = pd.read_csv(__scrobbles__, header = None, names = ['artist', 'album', 'song', 'datetime'])

In [11]:
# remove where datetime is NaN (negligible)
df = df[~df.datetime.isna()]

In [12]:
# convert datetime column to acutal datetime objects
# subtract 7 hours since original times are UTC
df.datetime = pd.to_datetime(df.datetime) - pd.Timedelta(hours = 7)

In [13]:
df.tail(20)

,artist,album,song,datetime
55538,Getter,Wat The Frick EP,Something New,2017-07-05 06:37:00
55539,Che Ecru,buries,2 AM,2017-07-05 06:34:00
55540,Deorro,Good Evening,Guide Me,2017-07-05 06:31:00
55541,BONJR,Hey Sego,hey sego,2017-07-05 06:28:00
55542,Oshi,oshi,state of flux,2017-07-05 06:24:00
55543,Young Thug,Slime Season 3,With Them,2017-07-05 06:20:00
55544,Hyper Potions,Adventures,Adventures,2017-07-04 20:53:00
55545,Young Thug,Beautiful Thugger Girls,Daddy's Birthday,2017-07-04 13:05:00
55546,Young Thug,Slime Season,Again,2017-07-04 13:01:00
55547,Young Thug,Slime Season,Overdosin,2017-07-04 12:50:00


# Time Analysis

In [32]:
df.datetime

1       2020-05-04 17:34:00
2       2020-05-04 15:07:00
3       2020-05-04 15:05:00
4       2020-05-04 15:05:00
5       2020-05-04 14:58:00
                ...        
55553   2017-07-04 12:26:00
55554   2017-07-04 12:23:00
55555   2017-07-01 14:33:00
55556   2017-06-29 20:39:00
55557   2017-06-29 20:34:00
Name: datetime, Length: 55281, dtype: datetime64[ns]

In [42]:
df.datetime.dt.weekday.map(weekday_map).value_counts()

thursday     9829
friday       9606
tuesday      9445
wednesday    7775
monday       7201
saturday     7029
sunday       4396
Name: datetime, dtype: int64

# Lyric Sentiment Analysis

In [44]:
# for genius API
# name           pysent
# URL            https://github.com/LeChase/spotify_hist
# client ID      lT9Ki6NGuPJC8bVLWtmNn0F2J5kYPiL0E7i9pHN8jHot6pnYb_x7rlFlbYAZepEV
# client secret  bElRrQIVRVRZRA3h3iPvUm2iSk6mpeQzYU8H_GPMsMAAJWgSHgSF2j_sigwjJVrvhdJ8d-StiZISXxNKD5cbog
# access token   UgvKlj9wHavgvBZScdTzfIKL2DKiIatIwPVV0M6JpEqm8YgIC_eJiQsYcsnmU2G-

In [14]:
import lyricsgenius, json

In [69]:
genius = lyricsgenius.Genius("UgvKlj9wHavgvBZScdTzfIKL2DKiIatIwPVV0M6JpEqm8YgIC_eJiQsYcsnmU2G-", timeout = 60, sleep_time = 2)

In [71]:
genius.verbose = False

In [17]:
# artist = genius.search_artist("Drake", max_songs=2,)

In [18]:
# artist.save_lyrics()

In [19]:
# # Reading the json as a dict
# with open('Lyrics_Drake.json') as json_data:
#     data = json.load(json_data)

In [20]:
# len(data['songs'])

In [22]:
# song = genius.search_song("somethign new getter")

In [56]:
def get_lyrics(row):
    try:
        song = genius.search_song(' - '.join((row.artist, row.song)))
        return song.lyrics
    except AttributeError:
        return 'NOT FOUND'

In [26]:
test = df.sample(20)

In [116]:
test.lyrics = test.apply(get_lyrics, axis = 1)

Searching for "Still Here Drake"...
Done.
Searching for "Life is Good (feat. Drake) Future"...
Done.
Searching for "Big Amount 2 Chainz"...
Done.
Searching for "Fuck That Check Up (feat. Lil Uzi Vert) Meek Mill"...
Done.
Searching for "Shelter Porter Robinson"...
Done.
Searching for "Rebound aftertheparty"...
Done.
Searching for "Check the Rhime A Tribe Called Quest"...
Done.
Searching for "No Time (feat. Gunna) Playboi Carti"...
Done.
Searching for "Find My Way DaBaby"...
Done.
Searching for "Paramedic! SOB x RBE"...
Done.
Searching for "Pursuit of Happiness (Nightmare) Kid Cudi"...
Done.
Searching for "200 Future"...
Done.
Searching for "Armed and Dangerous Juice WRLD"...
Done.
Searching for "South Galimatias"...
Done.
Searching for "No Role Modelz J. Cole"...
Done.
Searching for "BoomTrap Protocol Logic"...
Done.
Searching for "No Heart 21 Savage"...
Done.
Searching for "Feed Me Dope Future"...
Done.
Searching for "Lil Arrogant (feat. Joey Bada$$ & Russ) IDK"...
Done.
Searching for 

/Users/georgeschnaars/Desktop/Python/spotify_hist/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [118]:
test.lyrics

30513    [Intro]\nDoin' well dog\nYeah, me and all my\n...
7776     [Перевод песни "Future - Life Is Good (feat. D...
14649    [Intro: 2 Chainz]\nI'ma tell you, I'ma tell yo...
54424    [Intro: Meek Mill]\nLil' fish, yeah\nIf you ca...
7591     [Intro: Amy Millan]\nAyy, I will, ayy\nAyy, I ...
53000    [Pre-Chorus]\nBaby I know you want someone\nTo...
16181    [Intro]\nUh, uh, uh, uh\nUh, uh, uh, check the...
26555    [Intro: Playboi Carti]\nCheck it out\nWake up ...
551      [Intro]\nWait a minute, who are you? (Ayy, yo,...
36624    [Intro: Zacari & Kendrick Lamar]\nI am Killmon...
14215    [Intro: Kid Cudi]\nMmmm, ay-oh\nHey...\nRatata...
6283     [Intro: Young Thug]\nEverything right\nEveryth...
21353              \n            Lyrics for this song h...
41499    [Verse 1]\nLook me in the eye I'ma take it slo...
48604    [Produced by Phonix Beats and J. Cole]\n\n[Ver...
44165    [Verse 1]\nLet me get up in this bitch, like, ...
45620    [Intro]\nSouthside, Southside on the, Southsid.

In [121]:
test['lyrics'] = test.lyrics

In [122]:
test

,artist,album,song,datetime,lyrics
30513,Drake,Views,Still Here,2019-01-29 00:58:00,"[Intro]\nDoin' well dog\nYeah, me and all my\n..."
7776,Future,Life Is Good (feat. Drake),Life is Good (feat. Drake),2020-01-10 04:00:00,"[Перевод песни ""Future - Life Is Good (feat. D..."
14649,2 Chainz,Pretty Girls Like Trap Music,Big Amount,2019-10-10 13:41:00,"[Intro: 2 Chainz]\nI'ma tell you, I'ma tell yo..."
54424,Meek Mill,Wins & Losses,Fuck That Check Up (feat. Lil Uzi Vert),2017-07-25 05:29:00,"[Intro: Meek Mill]\nLil' fish, yeah\nIf you ca..."
7591,Porter Robinson,Shelter,Shelter,2020-01-13 06:15:00,"[Intro: Amy Millan]\nAyy, I will, ayy\nAyy, I ..."
53000,aftertheparty,?!,Rebound,2017-08-14 07:50:00,[Pre-Chorus]\nBaby I know you want someone\nTo...
16181,A Tribe Called Quest,The Low End Theory,Check the Rhime,2019-09-21 13:16:00,"[Intro]\nUh, uh, uh, uh\nUh, uh, uh, check the..."
26555,Playboi Carti,Die Lit,No Time (feat. Gunna),2019-03-29 13:57:00,[Intro: Playboi Carti]\nCheck it out\nWake up ...
551,DaBaby,Find My Way,Find My Way,2020-04-24 04:53:00,"[Intro]\nWait a minute, who are you? (Ayy, yo,..."
36624,SOB x RBE,Black Panther the Album Music From and Inspire...,Paramedic!,2018-10-04 08:30:00,[Intro: Zacari & Kendrick Lamar]\nI am Killmon...


In [157]:
%%timeit -r1
get_lyrics(test.iloc[0])

Searching for "Drake Still Here"...
Done.
Searching for "Drake Still Here"...
Done.
2.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [34]:
test_lyrics = get_lyrics(test.iloc[0])

In [35]:
len(test_lyrics)

3966

In [167]:
sys.getsizeof(test_lyrics)

4180

In [169]:
test['bytes'] = test.lyrics.apply(sys.getsizeof)

In [174]:
print(test.bytes.mean()*len(dupes)/1000000, 'MB total')

28.959085599999998 MB total


In [178]:
print(2.2 * len(dupes)/60/60, 'hrs to scrape')

4.6096111111111115 hrs to scrape


In [1]:
import util

In [2]:
help(util)

Help on module util:

NAME
    util - util module

FUNCTIONS
    suppress_stdout(func)
        decorator to redirect print output to null

FILE
    /Users/georgeschnaars/Desktop/Python/spotify_hist/util.py




In [3]:
help(util.suppress_stdout)

Help on function suppress_stdout in module util:

suppress_stdout(func)
    decorator to redirect print output to null



In [44]:
uniques = df.drop_duplicates(['artist', 'song'])[['artist', 'song']]

In [398]:
# print('starting:', time.ctime())
# uniques['lyrics'] = uniques.apply(get_lyrics, axis = 1)
# print('ending:', time.ctime())

In [110]:
test = uniques.groupby('artist')['song'].apply(list).reset_index(name = 'songs')

In [234]:
test

,artist,songs
0,!!!,[Even When the Water's Cold]
1,$uicideboy$,"[Either Hated Or Ignored, PARIS, REIGN IN BLOO..."
2,03 Greedo,"[Maria, Disco Shit, Beg Your Pardon, Soul Food..."
3,070 Shake,[Honey]
4,1-800 GIRLS,[All Day All Night]
...,...,...
2039,wasiu,[Cigarettes & Poutine]
2040,yugi boi,"[Peach Tea, The End, No Problem]"
2041,yuzu blur,[Sky Blush]
2042,ÊMIA,[Psychic]


In [235]:
d = test.to_dict('records')

In [237]:
# d

In [130]:
d = {item['artist']: item['songs'] for item in d}

In [132]:
len(d)

2044

In [136]:
# for artist, songs in d.items():
#     for song in songs:
#         print(artist, song)

In [137]:
import json

In [143]:
with open('uniques.json', 'w') as f:
    json.dump(d, f)

In [224]:
import collections

In [244]:
with open('uniques.json') as f:
    my_dict = json.load(f)

In [247]:
''

''

In [147]:
my_dict['TEST'] = ['hello']

In [179]:
my_dict['Drake'].append('tes5')

In [254]:
def dict_diff(dict_old, dict_new):
    # returns the items present in dict_b that are NOT present in dict_a
    return unflatten_dict(set(flatten_dict(dict_new)) - set(flatten_dict(dict_old)))

In [255]:
diff = dict_diff(d, my_dict)

AttributeError: 'list' object has no attribute 'items'

In [256]:
diff

{('Drake', 'tes5'), ('Drake', 'test'), ('TEST', 'hello')}

In [217]:
d = defaultdict(list)

In [218]:
for artist, song in diff:
    d[artist].append(song)

In [228]:
unflatten_dict(flatten_dict(d))

defaultdict(list, {'TEST': ['hello'], 'Drake': ['test', 'tes5']})

In [243]:
unflatten_dict(diff)

{'TEST': ['hello'], 'Drake': ['test', 'tes5']}

In [230]:
def flatten_dict(d):
    # ungroups keys
    return [(key, value) for key, values in d.items() for value in values]

In [242]:
def unflatten_dict(list_tups):
    # groups keys
    d = collections.defaultdict(list)
    for key, val in list_tups:
        d[key].append(val)
    return dict(d)

In [275]:
a = {1: ['efwgewg'], 2: ['ewrgerwg'], 5: ['iupup']}
b = {3: ['gewgweg'], 5: ['test']}

In [276]:
{**a, **b}

{1: ['efwgewg'], 2: ['ewrgerwg'], 5: ['test'], 3: ['gewgweg']}

In [277]:
{**b, **a}

{3: ['gewgweg'], 5: ['iupup'], 1: ['efwgewg'], 2: ['ewrgerwg']}

In [388]:
c = {'artist1': {'song1': '', 'song2': 'lyric'}, 'artist2': {'song1': 'lyric lyric'}, 'artist3': {'song1': 'lyric', 'song2': '', 'song3': 'howdy'}}

In [389]:
c

{'artist1': {'song1': '', 'song2': 'lyric'},
 'artist2': {'song1': 'lyric lyric'},
 'artist3': {'song1': 'lyric', 'song2': '', 'song3': 'howdy'}}

In [376]:
def get_empties(d):
    empties = defaultdict(dict)
    for key, sub_d in d.items():
        for sub_key, sub_val in sub_d.items():
            if not sub_val:
                empties[key][sub_key] = d[key][sub_key]
    return empties

In [377]:
def split_empties(d):
    empties = defaultdict(dict)
    filled = dict(d)
    for key, sub_d in d.items():
        print(key)
        for sub_key, sub_val in sub_d.items():
            print(sub_key)
            if not sub_val:
                empties[key][sub_key] = filled[key].pop(sub_key)
            print('done')
    return empties, d

In [378]:
split_empties(c)

artist1
song1
done


RuntimeError: dictionary changed size during iteration

In [392]:
filled = dict(c)

In [393]:
filled

{'artist1': {'song1': '', 'song2': 'lyric'},
 'artist2': {'song1': 'lyric lyric'},
 'artist3': {'song1': 'lyric', 'song2': '', 'song3': 'howdy'}}

In [395]:
filled['artist1'].pop('song1')

''

In [396]:
filled

{'artist1': {'song2': 'lyric'},
 'artist2': {'song1': 'lyric lyric'},
 'artist3': {'song1': 'lyric', 'song2': '', 'song3': 'howdy'}}

In [397]:
c

{'artist1': {'song2': 'lyric'},
 'artist2': {'song1': 'lyric lyric'},
 'artist3': {'song1': 'lyric', 'song2': '', 'song3': 'howdy'}}

In [399]:
import uniques

In [400]:
import pandas as pd

In [405]:
# json = pd.read_json("output/unique.json")
import json

In [612]:
with open("output/lyrics/unique_lyrics.json") as f:
    d = json.load(f)
    
with open("output/spotify/unique_spotify.json") as f_spot:
    d_spot = json.load(f_spot)

In [613]:
d_flat = uniques.Unique.flatten_dict(d)
d_spot_flat = uniques.Unique.flatten_dict(d_spot)

In [599]:
# d_spot_flat

In [676]:
df_lyrics = pd.DataFrame(d_flat, columns = ['artist', 'song', 'lyrics'])

In [648]:
df_spot = pd.DataFrame(d_spot_flat, columns = ['artist', 'song', 'features'])

In [649]:
# remove from list
df_spot.features = df_spot.features.apply(lambda x: x[0] if x!='' else '')

In [650]:
# groupby artist and song
df_spot = df_spot.groupby(['artist', 'song']).max()

In [651]:
# separate the data into 3 dataframes: filled with info, unable to be filled, and those yet to be checked
df_spot_filled = df_spot[df_spot.features.apply(type) == dict]
df_spot_unable = df_spot[df_spot.features.isna()]
df_spot_yet = df_spot[df_spot.features == ''].replace('', np.nan)

In [652]:
# confirm conservation of rows
len(df_spot) == len(df_spot_filled) + len(df_spot_unable) + len(df_spot_yet)

True

In [657]:
# # for completed rows
# # convert dictionaries into columns
df_spot_filled = pd.DataFrame(df_spot_filled.features.tolist(), index = df_spot_filled.index)
df_spot_filled.columns = [item + '_spotify' for item in df_spot_filled.columns]

In [660]:
# save completed data to json
df_spot_filled.to_json("output/spotify/completed.json", orient = 'table')

In [673]:
df_spot_unable.to_json("output/spotify/failed.json", orient = 'table')

In [674]:
df_spot_yet.to_json("output/spotify/yet.json", orient = 'table')

In [565]:
read_output.apply(lambda x: 'artist: {}, track: {}'.format(*x.name), axis = 1)

artist        song                                    
!!!           Even When the Water's Cold                     artist: !!!, track: Even When the Water's Cold
$uicideboy$   #1 STUNNA                                               artist: $uicideboy$, track: #1 STUNNA
              2ND HAND                                                 artist: $uicideboy$, track: 2ND HAND
              AM / PM                                                   artist: $uicideboy$, track: AM / PM
              Champion Of Death                               artist: $uicideboy$, track: Champion Of Death
                                                                                ...                        
Skrillex      HUMBLE. - SKRILLEX REMIX                    artist: Skrillex, track: HUMBLE. - SKRILLEX REMIX
              Purple Lamborghini (with Rick Ross)         artist: Skrillex, track: Purple Lamborghini (w...
              Scary Monsters and Nice Sprites             artist: Skrillex, track

In [682]:
(df_lyrics.head())

,artist,song,lyrics
0,!!!,Even When the Water's Cold,Friends told her she was better off\nAt the bo...
1,$uicideboy$,Either Hated Or Ignored,"[Intro: $LICK $LOTH]\nYah, yah, $carecrow, Rub..."
2,$uicideboy$,PARIS,"[Verse 1: $LICK $LOTH]\nAyy, ayy, ayy, tell me..."
3,$uicideboy$,REIGN IN BLOOD,"[Intro: Triggerman]\nDrag him from the river, ..."
4,$uicideboy$,Magazine,"[Intro: YUNG CHRIST]\n$uicidal holocaust, for ..."


In [683]:
# groupby artist and song
df_lyrics = df_lyrics.groupby(['artist', 'song']).max()

In [686]:
df_lyrics.lyrics.str.contains('NOT FOUND').value_counts()

False    6735
True      357
Name: lyrics, dtype: int64

In [695]:
# separate the data into 3 dataframes: filled with info, unable to be filled, and those yet to be checked
df_lyrics_filled = df_lyrics[~(df_lyrics.lyrics.str.contains('NOT FOUND') | (df_lyrics.lyrics == ''))]
df_lyrics_unable = df_lyrics[df_lyrics.lyrics.str.contains('NOT FOUND')].replace('NOT FOUND - AttributeError', np.nan)
df_lyrics_yet = df_lyrics[df_lyrics.lyrics == ''].replace('', np.nan)

In [696]:
# confirm conservation of rows
len(df_lyrics) == len(df_lyrics_filled) + len(df_lyrics_unable) + len(df_lyrics_yet)

True

In [861]:
df_lyrics_yet

lyrics
artist    song                           
Madeintyo 2 Flights                   NaN
          Cashing Out                 NaN
          Chucky Cheese               NaN
          Good Gas                    NaN
          I Want (feat. 2 Chainz)     NaN
...                                   ...
yugi boi  Peach Tea                   NaN
          The End                     NaN
yuzu blur Sky Blush                   NaN
ÊMIA      Psychic                     NaN
âme       No War - Rampa Remix        NaN

[3001 rows x 1 columns]

In [698]:
# save completed data to json
df_lyrics_filled.to_json("output/lyrics/completed.json", orient = 'table')

In [699]:
# save completed data to json
df_lyrics_unable.to_json("output/lyrics/failed.json", orient = 'table')

In [700]:
# save completed data to json
df_lyrics_yet.to_json("output/lyrics/yet.json", orient = 'table')

In [704]:
scrobbles_df = pd.read_csv(__scrobbles__, header = None, names = ['artist', 'album', 'song', 'datetime'])

In [705]:
scrobbles_df

,artist,album,song,datetime
0,Drake,Dark Lane Demo Tapes,Losses,NaN
1,Drake,Dark Lane Demo Tapes,Pain 1993 (with Playboi Carti),05 May 2020 00:34
2,Juice WRLD,Death Race For Love,Empty,04 May 2020 22:07
3,Juice WRLD,Death Race For Love,Rider,04 May 2020 22:05
4,Juice WRLD,Death Race For Love,Make Believe,04 May 2020 22:05
...,...,...,...,...
55553,Madeintyo,Thank You Mr. Tokyo,Time Of Her Life,04 Jul 2017 19:26
55554,DJ Khaled,Grateful,Iced Out My Arms,04 Jul 2017 19:23
55555,Young Thug,Beautiful Thugger Girls,Daddy's Birthday,01 Jul 2017 21:33
55556,Lil Wayne,Tha Carter IV (Deluxe),How to Love,30 Jun 2017 03:39


In [706]:
uniques_df = scrobbles_df.drop_duplicates(['artist', 'song'])[['artist', 'song']]

In [718]:
uniques_index = pd.MultiIndex.from_frame(uniques_df)

In [741]:
diff = set(uniques_index) - set(df_lyrics_filled.index) - set(df_lyrics_unable.index) - set()

In [742]:
len(diff), len(uniques_index), len(df_lyrics_filled), len(df_lyrics_unable), len(df_lyrics_yet)

(3001, 7092, 3734, 357, 3001)

In [746]:
# for artist, song in diff:
#     print(artist)

In [772]:
df = pd.DataFrame(index = diff)
df.index = df.index.set_names(['artist', 'song'])

In [775]:
df = df.sort_index()

In [819]:
df

test
artist    song                               
Madeintyo 2 Flights                       NaN
          Cashing Out                     NaN
          Chucky Cheese                   NaN
          Good Gas                      hello
          I Want (feat. 2 Chainz)         NaN
...                                       ...
yugi boi  The End                         NaN
yuzu blur Sky Blush                       NaN
ÊMIA      Psychic                         NaN
âme       No War - Rampa Remix            NaN
aaaa      Chucky                   mewghywion

[3002 rows x 1 columns]

In [829]:
test = pd.DataFrame(index = pd.MultiIndex(levels = [[]]*2, codes = [[]]*2, names = ['artist', 'song']))

In [831]:
test.index

MultiIndex([], names=['artist', 'song'])

In [839]:
# test.at[('aaaa', 'Chucky'), 'test'] = 'mewghywion'

In [805]:
import importlib

In [809]:
import lyrics

In [816]:
importlib.reload(lyrics)

<module 'lyrics' from '/Users/georgeschnaars/Desktop/Python/spotify_hist/lyrics.py'>

In [817]:
lyrics.Lyrics.test()

NameError: name 'failed_result' is not defined

In [841]:
placeholder = df.index

In [843]:
placeholder.drop(index = ('Drake', '10 Bands'))

TypeError: drop() got an unexpected keyword argument 'index'

In [862]:
# for i, (index, series) in enumerate(df.itertuples(), 1):
#     print(i, index)

In [866]:
df.drop(('Madeintyo', 'Cashing Out'))

test
artist    song                               
Madeintyo 2 Flights                       NaN
          Chucky Cheese                   NaN
          Good Gas                      hello
          I Want (feat. 2 Chainz)         NaN
          Mr. Tokyo                       NaN
...                                       ...
yugi boi  The End                         NaN
yuzu blur Sky Blush                       NaN
ÊMIA      Psychic                         NaN
âme       No War - Rampa Remix            NaN
aaaa      Chucky                   mewghywion

[3001 rows x 1 columns]

In [865]:
df

test
artist    song                               
Madeintyo 2 Flights                       NaN
          Cashing Out                     NaN
          Chucky Cheese                   NaN
          Good Gas                      hello
          I Want (feat. 2 Chainz)         NaN
...                                       ...
yugi boi  The End                         NaN
yuzu blur Sky Blush                       NaN
ÊMIA      Psychic                         NaN
âme       No War - Rampa Remix            NaN
aaaa      Chucky                   mewghywion

[3002 rows x 1 columns]

In [870]:
importlib.reload(uniques)

<module 'uniques' from '/Users/georgeschnaars/Desktop/Python/spotify_hist/uniques.py'>

In [875]:
pd.MultiIndex.from_tuples(uniques.Unique.get_uniques())

MultiIndex([(  'PARTYNEXTDOOR',                  'Loyal (feat. Drake)'),
            (          'Drake',                              'Jumpman'),
            ('Louis The Child',                         'It's strange'),
            (          'JAY-Z',                       'Why I Love You'),
            (         'Dwilly',                                 'ugh!'),
            (      'Giraffage',                                'Hello'),
            (          'Drake',            'Put It Down (feat. Drake)'),
            (       'A$AP Mob',             'Last Day of Skool (Skit)'),
            (          'Drake',                      'Say What's Real'),
            (      'Metallica', 'For Whom The Bell Tolls (Remastered)'),
            ...
            (  'Isaiah Rashad',                          'Cilvia Demo'),
            (       'SahBabii',                 'Titanoboa (feat. T3)'),
            (         'Maazel',                           'In My Head'),
            (       'Lil Baby',    